# Divisão e preparação do dataset
Esse notebook é responsável por:
- Remover as colunas description e url (colunas que não são úteis para a previsão do preço)
- Dividir o dataset em treino, validação e teste